In [1]:
import CG.x64.Debug.CG as CG
import unittest
import numpy as np
import time
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Label, LabelSet, Range1d
output_notebook()

Loading BokehJS ...

In [2]:
def gen_points(m: int) -> CG.Point2DVector:
    x = np.random.rand(m)
    y = np.random.rand(m)
    res = CG.Point2DVector()
    for i in range(len(x)):
        res.append(CG.Point2D(x[i], y[i]))
    return res

In [3]:
def draw_points_2d(points: CG.Point2DVector):
    f = figure()
    x = [p.x() for p in points]
    y = [p.y() for p in points]
    f.scatter(x, y)
    show(f)
    
def draw_line_2d(points: CG.Point2DVector):
    f = figure()
    x = [p.x() for p in points]
    y = [p.y() for p in points]
    text = [str(i) for i in range(len(x))]
    source = ColumnDataSource(data=dict(x=x, y=y, text=text))
    labels = LabelSet(x='x', y='y', text='text', text_font_size='10px', source=source, 
                      x_offset=3, y_offset=3, render_mode='canvas')
    f.scatter(x='x',y='y',source=source)
    f.line(x, y, line_dash=[4, 4], line_color="orange", line_width=1)
    f.add_layout(labels)
    show(f)

In [26]:
class TestGeomertry(unittest.TestCase):
    def test_in_triangle1(self):
        # 顺时针排序
        p = CG.Point2D(5.0, 0.0)
        q = CG.Point2D(0.0, 0.0)
        r = CG.Point2D(0.0, 5.0)
        self.assertFalse(CG.in_triangle(p, q, r, CG.Point2D(0.0, 0.0)))
        self.assertTrue(CG.in_triangle(p, q, r, CG.Point2D(1.0, 1.0)))
        self.assertFalse(CG.in_triangle(p, q, r, CG.Point2D(5.0, 5.0))) 
    def test_in_triangle2(self):
        # 逆时针排序
        p = CG.Point2D(5.0, 0.0)
        q = CG.Point2D(0.0, 5.0)
        r = CG.Point2D(0.0, 0.0)
        self.assertFalse(CG.in_triangle(p, q, r, CG.Point2D(0.0, 0.0)))
        self.assertTrue(CG.in_triangle(p, q, r, CG.Point2D(1.0, 1.0)))
        self.assertFalse(CG.in_triangle(p, q, r, CG.Point2D(5.0, 5.0)))
    def test_in_convex_polygon(self):
        convex_hull = CG.Point2DVector([
            CG.Point2D(0.0, 0.0),
            CG.Point2D(1.0, 0.0),
            CG.Point2D(2.0, 1.0),
            CG.Point2D(1.0, 2.0),
            CG.Point2D(0.0, 1.0)])
        self.assertFalse(CG.in_convex_polygon(convex_hull, CG.Point2D(0.0, 0.0)))
        self.assertTrue(CG.in_convex_polygon(convex_hull, CG.Point2D(0.5, 0.5)))
    def test_sort_by_angle(self):
        pts = gen_points(10)
        draw_line_2d(pts)
        ordered_pts = CG.sort_by_angle(CG.Point2D(0.0, 0.0), pts)
        draw_line_2d(ordered_pts)

In [28]:
res = unittest.main(argv=[''], verbosity=2, exit=False)

test_in_convex_polygon (__main__.TestGeomertry) ... ok
test_in_triangle1 (__main__.TestGeomertry) ... ok
test_in_triangle2 (__main__.TestGeomertry) ... ok
test_sort_by_angle (__main__.TestGeomertry) ... c:\Users\huangyan1\Desktop\computational_geometry\cgenv\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


ok

----------------------------------------------------------------------
Ran 4 tests in 0.281s

OK
